# BICS - Genome Analysis and Visualization

In [7]:
# Connect to Google Drive (make sure you have `metadata.tsv` in your Google Drive "My Drive" directory.)
'''from google.colab import drive
drive.mount('/content/drive')
'''

"from google.colab import drive\ndrive.mount('/content/drive')\n"

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from tqdm import tqdm
import re

###### metadata.tsv 733 MB

In [9]:
#data = pd.read_csv("/content/drive/MyDrive/metadata.tsv", sep="\t")
data = pd.read_csv("./files/metadata.tsv", sep="\t")

#with open("/content/drive/MyDrive/reference.gb") as f:
with open("./files/reference.gb") as f:
  reference_full_string = f.read()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (13,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
# Preview data
data.head(n=3)

,Virus name,Type,Accession ID,Collection date,Location,Additional location information,Sequence length,Host,Patient age,Gender,...,Pangolin version,Variant,AA Substitutions,Submission date,Is reference?,Is complete?,Is high coverage?,Is low coverage?,N-Content,GC-Content
0,hCoV-19/Australia/NT12/2020,betacoronavirus,EPI_ISL_426900,2020,Oceania / Australia / Northern territory,NaN,29862,Human,unknown,unknown,...,2021-05-19,NaN,"(NSP15_A283V,NSP12_P323L,Spike_D614G)",2020-04-17,NaN,True,True,NaN,0.006912,0.379674
1,hCoV-19/Australia/NT13/2020,betacoronavirus,EPI_ISL_426901,2020,Oceania / Australia / Northern territory,NaN,29865,Human,unknown,unknown,...,2021-05-19,NaN,"(NSP15_A283V,NSP12_P323L,Spike_D614G)",2020-04-17,NaN,True,True,NaN,0.008305,0.379554
2,hCoV-19/Australia/NT14/2020,betacoronavirus,EPI_ISL_426902,2020,Oceania / Australia / Northern territory,NaN,29864,Human,unknown,unknown,...,2021-05-19,NaN,"(NSP14_R163C,NSP3_K38R,NS3_G251V,NSP2_I559V,NS...",2020-04-17,NaN,True,NaN,NaN,0.017929,0.378860


In [11]:
len(data)

1755981

In [12]:
data.describe()

,Sequence length,N-Content,GC-Content
count,1.755981e+06,876815.000000,1.755981e+06
mean,2.931579e+04,0.033518,3.795408e-01
std,3.530986e+03,0.974207,2.624725e-03
min,6.400000e+01,0.000033,1.538462e-01
25%,2.976300e+04,0.000738,3.795925e-01
50%,2.978200e+04,0.008552,3.797707e-01
75%,2.982800e+04,0.028928,3.799121e-01
max,3.469200e+04,396.951613,5.131579e-01


In [13]:
full_data = data
data = full_data[::4]  # quarter of the entire dataset

# Reorganize Into Substitution Table
## Make a table (dataframe), where each row represents a single AA substitution

In [14]:
column_dict = {
    "name": "Virus name",
    "date": "Collection date",
    "age": "Patient age",
    "gender": "Gender",
    "version": "Pangolin version",
}

In [15]:
def parse_substitutions(subs):
  """
  Arguments
  - subs: "(NSP15_A283V,NSP12_P323L,Spike_D614G)"
  Returns
  - List of individual substitutions: [
    ("NSP15_A283V", "NSP15", "A", "V", 283),
    ...
  ]
  """
  if pd.isna(subs):  # nan value
    return []

  subs = subs.replace("(", "").replace(")", "")
  subs = subs.split(",")
  result = []
  for s in subs:
    try:
      feature, mut = s.split("_")
      before = mut[0]
      after = mut[-1]
      index = int(mut[1:-1])
    except:  # other types of substitutions
      continue
    result.append((s, feature, before, after, index))
  return result

In [17]:
sub_data = defaultdict(list)
for i, row in tqdm(data.iterrows(), total=len(data)):
  # Identify mutations
  subs = parse_substitutions(row["AA Substitutions"])
  location_info = [s.strip() for s in row["Location"].split("/")]
  continent = location_info[0]
  country = location_info[1]
  city = None
  territory = None
  if len(location_info) > 2:
    territory = location_info[2]
    if len(location_info) > 3:
      city = location_info[3]

  # Copy over columns from the column dict
  for new_column, old_column in column_dict.items():
    sub_data[new_column].extend([row[old_column]] * len(subs))

  for full_sub, feature, before, after, index in subs:
    # Fill mutation columns
    sub_data["sub"].append(full_sub)
    sub_data["sub_feature"].append(feature)
    sub_data["sub_before"].append(before)
    sub_data["sub_after"].append(after)
    sub_data["sub_index"].append(index)

    # Fill location columns
    sub_data["loc_continent"].append(continent)
    sub_data["loc_country"].append(country)
    sub_data["loc_territory"].append(territory)
    sub_data["loc_city"].append(city)

sub_df = pd.DataFrame(sub_data)
sub_df.head()

100%|████████████████████████████████████████████████████████████████████████| 438996/438996 [03:26<00:00, 2130.40it/s]


,name,date,age,gender,version,sub,sub_feature,sub_before,sub_after,sub_index,loc_continent,loc_country,loc_territory,loc_city
0,hCoV-19/Australia/NT12/2020,2020,unknown,unknown,2021-05-19,NSP15_A283V,NSP15,A,V,283,Oceania,Australia,Northern territory,None
1,hCoV-19/Australia/NT12/2020,2020,unknown,unknown,2021-05-19,NSP12_P323L,NSP12,P,L,323,Oceania,Australia,Northern territory,None
2,hCoV-19/Australia/NT12/2020,2020,unknown,unknown,2021-05-19,Spike_D614G,Spike,D,G,614,Oceania,Australia,Northern territory,None
3,hCoV-19/Australia/NT17/2020,2020,unknown,unknown,2021-05-19,NSP12_P323L,NSP12,P,L,323,Oceania,Australia,Northern territory,None
4,hCoV-19/Australia/NT17/2020,2020,unknown,unknown,2021-05-19,Spike_D614G,Spike,D,G,614,Oceania,Australia,Northern territory,None


In [18]:
len(data)

438996

In [19]:
len(sub_df)

6557926

In [20]:
sub_df.loc_city.unique()

array([None, 'Sydney', 'Rotterdam', ..., 'Lot', 'Tarn-et-Garonne', 'Gers'],
      dtype=object)

In [21]:
sub_df["sub"].head()

0    NSP15_A283V
1    NSP12_P323L
2    Spike_D614G
3    NSP12_P323L
4    Spike_D614G
Name: sub, dtype: object

In [22]:
unique_subs = sub_df["sub"].unique().tolist()
unique_subs[::5000]

['NSP15_A283V',
 'NSP9_A108T',
 'NSP9_D50B',
 'NSP6_T20A',
 'NS3_I7F',
 'NSP6_E250D',
 'N_D415V',
 'NSP3_G1423D',
 'NSP3_E1733K']

In [23]:
len(unique_subs)

40592

In [24]:
sub_df["sub"].value_counts()

Spike_D614G    427632
NSP12_P323L    416431
N_R203K        247788
N_G204R        233851
Spike_N501Y    201407
                ...  
NSP12_A125T         1
NSP15_G156C         1
Spike_N928D         1
NS8_P85Q            1
NSP12_A376C         1
Name: sub, Length: 40592, dtype: int64

In [25]:
print(sorted(sub_df["sub_feature"].unique().tolist()))

['E', 'M', 'N', 'NS3', 'NS6', 'NS7a', 'NS7b', 'NS8', 'NSP1', 'NSP10', 'NSP12', 'NSP13', 'NSP14', 'NSP15', 'NSP16', 'NSP2', 'NSP3', 'NSP4', 'NSP5', 'NSP6', 'NSP7', 'NSP8', 'NSP9', 'Spike']


# Mutation Index
Let's figure out where each mutation occured, within the full genome sequence. Let's look at this key: NSP12_D291N.

This means that the mutation occured in NSP12 and the exact location is index 291 (relative to the start of NSP12).

We can use this information to find the exact location of each substituion.

## Find Feature Locations From GenBank File
We need to find the start index of each feature (including proteins, peptides, etc.).

In [26]:
def parse_feature_locations(genbank_path):
  with open(genbank_path) as f:
    gb = f.read()
  gb_oneline = gb.replace("\n", "")
  gen_regex = r'gene.+?(\d*)\.\.(\d*).+?\/gene="(.+?)"'
  pep_regex = r'mat_peptide.+?(\d*)\.\.(\d*).+?/gene="(.+?)".+?/product="(.+?)".+?/note="(.{10})'
  gen_data = re.findall(gen_regex, gb_oneline)
  pep_data = re.findall(pep_regex, gb_oneline)

  # Gene Ranges
  range_dict = dict()
  letters = ["E", "M", "N", "S"]
  for start, end, name in gen_data:
      start = int(start)
      for letter in letters:
          if letter in name:
              range_dict[name] = (start - 1, end)

  # Peptide Ranges
  candidates = []
  for i in range(1, 17):
      candidates.append("nsp{}".format(i))
  candidates.reverse()  # e.g., check nsp11 before nsp1
  for start, end, _, name, note in pep_data:
      start = int(start)
      end = int(end)
      for c in candidates:
          if c in name.lower() or c in note.lower():
              range_dict[c.upper()] = (start - 1, end)
              
  # Spike Protein Ranges
  range_dict["Spike"] = range_dict["S"]
  del range_dict["S"]

  return dict(range_dict)

In [27]:
#range_dict = parse_feature_locations("/content/drive/MyDrive/reference.gb")
range_dict = parse_feature_locations("./files/reference.gb")

print(range_dict)

{'E': (26244, '26472'), 'M': (26522, '27191'), 'N': (28273, '29533'), 'NSP1': (13441, 13480), 'NSP2': (805, 2719), 'NSP3': (2719, 8554), 'NSP4': (8554, 10054), 'NSP5': (10054, 10972), 'NSP6': (10972, 11842), 'NSP7': (11842, 12091), 'NSP8': (12091, 12685), 'NSP9': (12685, 13024), 'NSP10': (13024, 13441), 'NSP12': (13441, 13468), 'NSP13': (16236, 18039), 'NSP14': (18039, 19620), 'NSP15': (19620, 20658), 'NSP16': (20658, 21552), 'NSP11': (13441, 13480), 'Spike': (21562, '25384')}


## Filter Features
### Filter rows that contain features for which we have the starting index

In [28]:
full_sub_df = sub_df

sub_df = sub_df[sub_df["sub_feature"].isin(range_dict.keys())]
sub_df

,name,date,age,gender,version,sub,sub_feature,sub_before,sub_after,sub_index,loc_continent,loc_country,loc_territory,loc_city
0,hCoV-19/Australia/NT12/2020,2020,unknown,unknown,2021-05-19,NSP15_A283V,NSP15,A,V,283,Oceania,Australia,Northern territory,None
1,hCoV-19/Australia/NT12/2020,2020,unknown,unknown,2021-05-19,NSP12_P323L,NSP12,P,L,323,Oceania,Australia,Northern territory,None
2,hCoV-19/Australia/NT12/2020,2020,unknown,unknown,2021-05-19,Spike_D614G,Spike,D,G,614,Oceania,Australia,Northern territory,None
3,hCoV-19/Australia/NT17/2020,2020,unknown,unknown,2021-05-19,NSP12_P323L,NSP12,P,L,323,Oceania,Australia,Northern territory,None
4,hCoV-19/Australia/NT17/2020,2020,unknown,unknown,2021-05-19,Spike_D614G,Spike,D,G,614,Oceania,Australia,Northern territory,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6557921,hCoV-19/Croatia/1692/2021,2021-03-12,70,Male,2021-05-27,NSP12_P323L,NSP12,P,L,323,Europe,Croatia,Požega Slavonia County,None
6557922,hCoV-19/Croatia/1692/2021,2021-03-12,70,Male,2021-05-27,Spike_D614G,Spike,D,G,614,Europe,Croatia,Požega Slavonia County,None
6557923,hCoV-19/Croatia/1692/2021,2021-03-12,70,Male,2021-05-27,N_D3L,N,D,L,3,Europe,Croatia,Požega Slavonia County,None
6557924,hCoV-19/Croatia/1692/2021,2021-03-12,70,Male,2021-05-27,Spike_S982A,Spike,S,A,982,Europe,Croatia,Požega Slavonia County,None


## Add Mutation Index To Table
Add the following columns based on the ranges obtained above:

* sub_feature_start
* sub_full_index

In [29]:
start_dict = {key: value[0] for key, value in range_dict.items()}
map_start_index = (lambda feature: start_dict[feature])

sub_df["sub_feature_start"] = sub_df["sub_feature"].apply(map_start_index)
sub_df["sub_full_index"] = sub_df["sub_feature_start"] + sub_df["sub_index"] * 3
# assumes that sub index is the AA index

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [30]:
sub_df

,name,date,age,gender,version,sub,sub_feature,sub_before,sub_after,sub_index,loc_continent,loc_country,loc_territory,loc_city,sub_feature_start,sub_full_index
0,hCoV-19/Australia/NT12/2020,2020,unknown,unknown,2021-05-19,NSP15_A283V,NSP15,A,V,283,Oceania,Australia,Northern territory,None,19620,20469
1,hCoV-19/Australia/NT12/2020,2020,unknown,unknown,2021-05-19,NSP12_P323L,NSP12,P,L,323,Oceania,Australia,Northern territory,None,13441,14410
2,hCoV-19/Australia/NT12/2020,2020,unknown,unknown,2021-05-19,Spike_D614G,Spike,D,G,614,Oceania,Australia,Northern territory,None,21562,23404
3,hCoV-19/Australia/NT17/2020,2020,unknown,unknown,2021-05-19,NSP12_P323L,NSP12,P,L,323,Oceania,Australia,Northern territory,None,13441,14410
4,hCoV-19/Australia/NT17/2020,2020,unknown,unknown,2021-05-19,Spike_D614G,Spike,D,G,614,Oceania,Australia,Northern territory,None,21562,23404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6557921,hCoV-19/Croatia/1692/2021,2021-03-12,70,Male,2021-05-27,NSP12_P323L,NSP12,P,L,323,Europe,Croatia,Požega Slavonia County,None,13441,14410
6557922,hCoV-19/Croatia/1692/2021,2021-03-12,70,Male,2021-05-27,Spike_D614G,Spike,D,G,614,Europe,Croatia,Požega Slavonia County,None,21562,23404
6557923,hCoV-19/Croatia/1692/2021,2021-03-12,70,Male,2021-05-27,N_D3L,N,D,L,3,Europe,Croatia,Požega Slavonia County,None,28273,28282
6557924,hCoV-19/Croatia/1692/2021,2021-03-12,70,Male,2021-05-27,Spike_S982A,Spike,S,A,982,Europe,Croatia,Požega Slavonia County,None,21562,24508


## Congrats!
We've successfully pre-processed the data into the substitution table. This should help us analyze and visualize stats, related to gene substitutions, much more easily.